<a href="https://colab.research.google.com/github/karthik-karalgikar/Aedes/blob/main/unet%2B%2Bipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
# The path below should point to the directory containing this notebook and the associated utility files
# Change it if necessary
os.chdir('/content/gdrive/MyDrive/Dataset_Segmentation')
!ls
PATH='/content/drive/My Drive/Dataset_Segmentation/Training'

Testing  Training  Untitled1.ipynb


In [4]:
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
#from tensorflow import keras
import tensorflow as tf

In [6]:
from model_new import *
img_size=(256,256)

In [7]:
n_class=2

In [8]:
mod = Nest_Net(img_size[0],img_size[1],1,1,deep_supervision=True)

In [28]:
import os
import numpy as np
from PIL import Image

# Function to generate mask images for all images in a folder
def generate_masks(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through training and testing folders
    for split_folder in os.listdir(input_folder):
        split_folder_path = os.path.join(input_folder, split_folder)

        # Skip over any non-directory files
        if not os.path.isdir(split_folder_path):
            continue

        # Iterate through each class folder (Aedes and non-aedes)
        for class_name in os.listdir(split_folder_path):
            class_folder = os.path.join(split_folder_path, class_name)

            # Skip over any non-directory files
            if not os.path.isdir(class_folder):
                continue

            # Iterate through each image in the class folder
            for filename in os.listdir(class_folder):
                # Skip over any non-image files
                if not filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    continue

                # Load the input image
                image_path = os.path.join(class_folder, filename)
                try:
                    image = Image.open(image_path)
                except Exception as e:
                    print(f"Error: {e} - Skipping file: {image_path}")
                    continue

                # Determine the class label and generate the corresponding mask
                if class_name == 'Aedes':
                    mask = np.ones_like(image) * 255  # For Aedes, assign all pixels as 255 (white)
                elif class_name == 'Non Aedes':
                    mask = np.zeros_like(image)  # For Non Aedes, assign all pixels as 0 (black)

                # Convert mask to PIL Image
                mask_image = Image.fromarray(mask.astype(np.uint8))

                # Save the mask image
                mask_filename = os.path.splitext(filename)[0] + "_mask.png"  # Example: input_image.png -> input_image_mask.png
                mask_image_path = os.path.join(output_folder, split_folder, class_name, mask_filename)
                os.makedirs(os.path.join(output_folder, split_folder, class_name), exist_ok=True)
                mask_image.save(mask_image_path)

# Example usage
input_folder = "/content/gdrive/MyDrive/Dataset_Segmentation"
output_folder = "/content/gdrive/MyDrive/Mask_Segmentation"
generate_masks(input_folder, output_folder)


In [21]:
!ls "/content/gdrive/MyDrive"

'Colab Notebooks'   Dataset_Segmentation


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import skimage.io as io
import skimage.transform as trans
import matplotlib.pyplot as plt
import scipy.misc as sc


def adjustData(img,mask,flag_multi_class,n_class):

    if(flag_multi_class):
        img /= 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (n_class,))
        for i in range(n_class):

            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img)>1):
        img = img / 255
        mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <=0.5] = 0
        #print(np.shape(mask),np.shape(img))
    return (img,mask)

def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,n_class = n_class,save_to_dir = None,target_size = img_size,seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        #'./',
        train_path,
        classes = [image_folder],
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        class_mode=None,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        class_mode=None,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,n_class)
        yield (img,mask)

def testGenerator(test_path,target_size = img_size,flag_multi_class = True,as_gray = True):
    files=sorted(os.listdir(test_path))
    num_image=len(files)
    for i in range(num_image):
        img = io.imread(os.path.join(test_path,files[i]),as_gray = as_gray)
        print(files[i])
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        #print(np.max(img))
        yield img

In [11]:
def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i] = color_dict[i]

    return img_out


def saveResult(img_path,save_path,npyfile,flag_multi_class = False,num_class = 2):
    files=os.listdir(img_path)
    #print(len(img_path))
    #print(len(npyfile))

    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        #img1=np.array(((img - np.min(img))/np.ptp(img))>0.6).astype(float)
        img[img>0.5]=1
        img[img<=0.5]=0
        io.imsave(os.path.join(save_path, files[i]),img)



def SaveResultwImage(img_path,save_path,npyfile,target_size=img_size,flag_multi_class = False,num_class = 2):
    files=os.listdir(img_path)
    #print(len(img_path))
    #print(len(npyfile))

    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        #img1=np.array(((img - np.min(img))/np.ptp(img))>0.6).astype(float)
        img[img>0.2]=1
        img[img<=0.2]=0

        I = io.imread(os.path.join(img_path,files[i]), as_gray=True)
        I = trans.resize(I,target_size)
        #dst = cv2.addWeighted(img, 0.5, I, 0.5, 0.0)
        img=np.true_divide((I+img),2)
        io.imsave(os.path.join(save_path, files[i]),img)

In [47]:
import sklearn.metrics as sm

def get_confusion_matrix_elements(groundtruth_list, predicted_list):
    """returns confusion matrix elements i.e TN, FP, FN, TP as floats
	See example code for helper function definitions
    """
    tn, fp, fn, tp = sm.confusion_matrix(groundtruth_list, predicted_list,labels=[0,1]).ravel()
    tn, fp, fn, tp = np.float64(tn), np.float64(fp), np.float64(fn), np.float64(tp)

    return tn, fp, fn, tp

def get_prec_rec_IoU_accuracy(groundtruth_list, predicted_list):
    """returns precision, recall, IoU and accuracy metrics
	"""
    tn, fp, fn, tp = get_confusion_matrix_elements(groundtruth_list, predicted_list)

    total = tp + fp + fn + tn
    accuracy = (tp + tn) / total
    prec=tp/(tp+fp)
    rec=tp/(tp+fn)
    IoU=tp/(tp+fp+fn)

    return prec,rec,IoU,accuracy

def get_f1_score(groundtruth_list, predicted_list):
    """Return f1 score covering edge cases"""

    tn, fp, fn, tp = get_confusion_matrix_elements(groundtruth_list, predicted_list)

    f1_score = (2 * tp) / ((2 * tp) + fp + fn)

    return f1_score

def get_validation_metrics(groundtruth,predicted):
    """Return all output metrics. Input is binary images"""

    u,v=np.shape(groundtruth)
    groundtruth_list=np.reshape(groundtruth,(u*v,))
    predicted_list=np.reshape(predicted,(u*v,))
    prec,rec,IoU,acc=get_prec_rec_IoU_accuracy(groundtruth_list, predicted_list)
    f1_score=get_f1_score(groundtruth_list, predicted_list)
   # print("Precision=",prec, "Recall=",rec, "IoU=",IoU, "acc=",acc, "F1=",f1_score)
    return prec,rec,IoU,acc,f1_score

def evalResult(gth_path,npyfile,target_size=img_size,flag_multi_class = False,num_class = 2):
    files=sorted(os.listdir(gth_path))
    #print(files)
    prec=0
    rec=0
    acc=0
    IoU=0
    f1_score=0
    for i,item in enumerate(npyfile):
        img = item[:,:,0]
        gth = io.imread(os.path.join(gth_path,files[i]))
        gth = trans.resize(gth,target_size)
        if(np.sum(img)>0):
            img=np.array(((img - np.min(img))/np.ptp(img))>0.1).astype(float)
        if(np.sum(gth)>0):
            gth=np.array(((gth - np.min(gth))/np.ptp(gth))>0.1).astype(float)
            gth=(gth>0.1).astype(int)
        #print(np.shape(gth),np.shape(img))
        p,r,I,a,f=get_validation_metrics(gth[:,:,1],img)
        if (np.isnan(p)):
          p=1
        if (np.isnan(r)):
          r=1
        if (np.isnan(f)):
          f=1
        if (np.isnan(I)):
          I=1
        prec=prec+p
        rec=rec+r
        acc=acc+a
        IoU=IoU+I
        f1_score=f1_score+f
    print("Precision=",prec/(i+1), "Recall=",rec/(i+1), "Jac=",IoU/(i+1), "acc=",acc/(i+1), "Dice=",f1_score/(i+1))

In [14]:
os.chdir('/content/gdrive/MyDrive/Dataset_Segmentation/Training')
data_gen_args = dict(rotation_range=0.2,
                     #rescale=1./255,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.1,
                    zoom_range=[0.8,1],
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')
PATH='./train/'

In [51]:
data_gen = trainGenerator(3,PATH,'/content/gdrive/MyDrive/Dataset_Segmentation/Training/Aedes','/content/gdrive/MyDrive/Mask_Segmentation/Training/Aedes',data_gen_args, save_to_dir = None)

In [52]:
mod.compile(optimizer = Adam(learning_rate=0.0003), loss = dice_coef_loss, metrics = dice_coef)

In [53]:
hist = mod.fit(data_gen,steps_per_epoch=10,epochs=80,verbose=1)

KeyError: 'Aedes'